In [ ]:
import multiprocessing
from functools import partial
import numpy as np
# from numba import jit

In [ ]:
#TODO: loss of least square regression and binary logistic regression
'''
    pred() takes GBDT/RF outputs, i.e., the "score", as its inputs, and returns predictions.
    g() is the gradient/1st order derivative, which takes true values "true" and scores as input, and returns gradient.
    h() is the heassian/2nd order derivative, which takes true values "true" and scores as input, and returns hessian.
'''
class leastsquare(object):
    '''Loss class for mse. As for mse, pred function is pred=score.'''
    def pred(self,score):
        return score

    def g(self,true,score):
      return 2 * (score - true)
      pass

    def h(self,true,score):
      return 2 * np.ones_like(score)
      pass

class logistic(object):
    '''Loss class for log loss. As for log loss, pred function is logistic transformation.'''
    def pred(self,score):
      return 1 / (1 + np.exp(-score))
      pass

    def g(self,true,score):
      return self.pred(score) - true
      pass

    def h(self,true,score):
      p = self.pred(score)
      return p * (1 - p)

In [ ]:
# TODO: class of Random Forest

# Import mode function
from scipy.stats import mode

class RF(object):
    '''
    Class of Random Forest

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth d_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10,
        lamda = 1, gamma = 0,
        rf = 0.99, num_trees = 100):

        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.num_trees = num_trees

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO
        self.trees = []
        n_samples, n_features = train.shape
        for _ in range(self.num_trees):
            # Bootstrap sample
            indices = np.random.choice(n_samples, n_samples, replace=True)
            train_sample = train[indices]
            target_sample = target[indices]
            # Train a tree
            tree = Tree(
                n_threads=self.n_threads,
                max_depth=self.max_depth,
                min_sample_split=self.min_sample_split,
                lamda=self.lamda,
                gamma=self.gamma,
                rf=self.rf,
                loss=self.loss  # Pass the loss function
            )
            tree.fit(train_sample, target_sample)
            self.trees.append(tree)
        return self

    def predict(self, test):
        # Aggregate predictions from all trees
        predictions = np.array([tree.predict(test) for tree in self.trees])
        if self.loss == 'log':
            # Majority voting for classification
            pred_labels, _ = mode(predictions, axis=0)
            return pred_labels[0]
        else:
            # Average for regression
            return np.mean(predictions, axis=0)

In [ ]:
# TODO: class of GBDT
class GBDT(object):
    '''
    Class of gradient boosting decision tree (GBDT)

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth D_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        learning_rate: The learning rate eta of GBDT.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10,
        lamda = 1, gamma = 0,
        learning_rate = 0.1, num_trees = 100):

        self.n_threads = n_threads
        if loss == 'mse':
            self.loss = leastsquare()
        elif loss == 'log':
            self.loss = logistic()
        else:
            self.loss = loss  # Assume it's a loss object

        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.num_trees = num_trees
        self.loss_name = loss

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO
        self.trees = []
        n_samples = train.shape[0]
        # Initialize predictions to zero
        self.score = np.zeros(n_samples)
        for _ in range(self.num_trees):
            # Compute gradients and hessians
            g = self.loss.g(target, self.score)
            h = self.loss.h(target, self.score)
            # Fit a tree to the negative gradients
            tree = Tree(
                n_threads=self.n_threads,
                max_depth=self.max_depth,
                min_sample_split=self.min_sample_split,
                lamda=self.lamda,
                gamma=self.gamma,
                loss=self.loss_name
            )
            tree.fit(train, None, g, h)
            # Update the scores
            pred = tree.predict(train)
            self.score += self.learning_rate * pred
            self.trees.append(tree)
        return self

    def predict(self, test):
        #TODO
        # Initialize predictions to zero
        score = np.zeros(test.shape[0])
        for tree in self.trees:
            pred = tree.predict(test)
            score += self.learning_rate * pred
        if self.loss_name == logistic():
          return (self.loss.pred(score)>0.5).astype(int)
        else:
          return self.loss.pred(score)

In [ ]:
# TODO: class of a node on a tree
class TreeNode(object):
    '''
    Data structure that are used for storing a node on a tree.

    A tree is presented by a set of nested TreeNodes,
    with one TreeNode pointing two child TreeNodes,
    until a tree leaf is reached.

    A node on a tree can be either a leaf node or a non-leaf node.
    '''

    #TODO
    def __init__(self, is_leaf=False, prediction=None, feature_index=None, threshold=None,
                 left_child=None, right_child=None):
        self.is_leaf = is_leaf
        self.prediction = prediction  # Value to predict if leaf node
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold  # Threshold value to split at
        self.left_child = left_child  # Left child node
        self.right_child = right_child  # Right child node


In [ ]:
# TODO: class of single tree
class Tree(object):
    '''
    Class of a single decision tree in GBDT or RF

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        max_depth: The maximum depth of the tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf prediction, also known as lambda.
        gamma: The regularization coefficient for number of TreeNode, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule,
            rf = 0 means we are training a GBDT.
    '''

    def __init__(self, n_threads = None,
                 max_depth = 3, min_sample_split = 10,
                 lamda = 1, gamma = 0, rf = 0, loss = 'mse'):
        self.n_threads = n_threads
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.int_member = 0
        self.loss = loss

        self.feature_indices = None  # To store the subset of features used
        self.root = None

    def fit(self, train, target, g=None, h=None):
        '''
        train is the training data matrix, and must be numpy array (an n_train x m matrix).
        g and h are gradient and hessian respectively.
        '''
        #TODO
        if g is not None and h is not None:
            self.is_gbdt = True
            # print("GBDT",self.is_gbdt)
        else:
            self.is_gbdt = False
            #print("GBDT",self.is_gbdt)
        if not self.is_gbdt and self.loss == 'log':
            self.is_classification = True
            #print("RF and classification",self.is_classification)
        else:
            self.is_classification = False
            # print("RF and classification",self.is_classification)
        n_samples, n_features = train.shape
        if self.rf > 0:
            m = n_features
            k = max(1, int(self.rf * m))
            self.feature_indices = np.random.choice(m, k, replace=False)
        else:
            self.feature_indices = np.arange(n_features)
        self.root = self.construct_tree(train, target, g, h, depth=0)
        return self

    def predict(self,test):
        '''
        test is the test data matrix, and must be numpy arrays (an n_test x m matrix).
        Return predictions (scores) as an array.
        '''
        #TODO
        result = np.array([self._predict_row(x, self.root) for x in test])
        return result

    def _predict_row(self, x, node):
      if node.is_leaf:
          return node.prediction
      else:
          if x[node.feature_index] <= node.threshold:
              return self._predict_row(x, node.left_child)
          else:
              return self._predict_row(x, node.right_child)

    def construct_tree(self, train, target, g, h, depth):
        '''
        Tree construction, which is recursively used to grow a tree.
        First we should check if we should stop further splitting.

        The stopping conditions include:
            1. tree reaches max_depth $d_{max}$
            2. The number of sample points at current node is less than min_sample_split, i.e., $n_{min}$
            3. gain <= 0
        '''
        #TODO
        n_samples, n_features = train.shape
        if depth >= self.max_depth or n_samples <= self.min_sample_split:
            # Make a leaf node
            if self.is_gbdt:
                G = np.sum(g)
                H = np.sum(h)
                pred = -G / (H + self.lamda)
            elif self.is_classification:
                # Predict the majority class
                counts = np.bincount(target)
                pred = np.argmax(counts)
            else:
                pred = np.mean(target)
            return TreeNode(is_leaf=True, prediction=pred)
        else:
            # Find the best decision rule
            feature, threshold, gain = self.find_best_decision_rule(train, target, g, h)
            if gain <= 0 or feature is None:
                # Make a leaf node
                if self.is_gbdt:
                    G = np.sum(g)
                    H = np.sum(h)
                    pred = -G / (H + self.lamda)
                elif self.is_classification:
                    # Predict the majority class
                    counts = np.bincount(target)
                    pred = np.argmax(counts)
                else:
                    pred = np.mean(target)
                return TreeNode(is_leaf=True, prediction=pred)
            else:
                # Split the data
                idx_left = train[:, feature] <= threshold
                idx_right = train[:, feature] > threshold
                if self.is_gbdt:
                    left_child = self.construct_tree(train[idx_left], None, g[idx_left], h[idx_left], depth + 1)
                    right_child = self.construct_tree(train[idx_right], None, g[idx_right], h[idx_right], depth + 1)
                else:
                    left_child = self.construct_tree(train[idx_left], target[idx_left], None, None, depth + 1)
                    right_child = self.construct_tree(train[idx_right], target[idx_right], None, None, depth + 1)
                return TreeNode(
                    is_leaf=False,
                    feature_index=feature,
                    threshold=threshold,
                    left_child=left_child,
                    right_child=right_child
                )

        # return TreeNode(split_feature = feature, split_threshold = threshold,
        #                 left_child = left_child, right_child = right_child)

    def find_best_decision_rule(self, train, target, g, h):
        '''
        Return the best decision rule [feature, treshold], i.e., $(p_j, \tau_j)$ on a node j,
        train is the training data assigned to node j
        g and h are the corresponding 1st and 2nd derivatives for each data point in train
        g and h should be vectors of the same length as the number of data points in train

        for each feature, we find the best threshold by find_threshold(),
        a [threshold, best_gain] list is returned for each feature.
        Then we select the feature with the largest best_gain,
        and return the best decision rule [feature, treshold] together with its gain.
        '''
        #TODO
        n_samples, n_features = train.shape
        # Prepare data for multiprocessing
        is_gbdt = self.is_gbdt
        is_classification = self.is_classification
        lamda = self.lamda
        gamma = self.gamma

        # Prepare arguments for parallel processing
        args_list = []
        for feature in self.feature_indices:
            feature_values = train[:, feature]
            args = (feature, feature_values, target, g, h, is_gbdt, is_classification, lamda, gamma)
            args_list.append(args)

        # Determine the number of processes
        if self.n_threads is None:
            num_processes = multiprocessing.cpu_count()
        else:
            num_processes = self.n_threads

        # Use multiprocessing to find the best thresholds for each feature in parallel
        with multiprocessing.Pool(processes=num_processes) as pool:
            results = pool.map(find_best_for_feature, args_list)

        # Select the best feature and threshold based on gain
        best_gain = -np.inf
        best_feature = None
        best_threshold = None
        for feature, threshold, gain in results:
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold

        return best_feature, best_threshold, best_gain

# Define the function for parallel computation
def find_best_for_feature(args):
    feature, feature_values, target, g, h, is_gbdt, is_classification, lamda, gamma = args
    threshold, gain = find_threshold_static(feature_values, target, g, h, is_gbdt, is_classification, lamda, gamma)
    return feature, threshold, gain

# Define the static method for finding the best threshold
def find_threshold_static(feature_values, target, g, h, is_gbdt, is_classification, lamda, gamma):
    sorted_idx = np.argsort(feature_values)
    feature_values = feature_values[sorted_idx]
    if is_gbdt:
        g = g[sorted_idx]
        h = h[sorted_idx]
        G_total = np.sum(g)
        H_total = np.sum(h)
        G_left = 0.0
        H_left = 0.0
    elif is_classification:
        target = target[sorted_idx]
        n_total = len(target)
        # Compute Gini impurity of the parent node
        classes, counts = np.unique(target, return_counts=True)
        p_classes = counts / n_total
        gini_parent = 1.0 - np.sum(p_classes ** 2)
    else:
        target = target[sorted_idx]
        total_variance = np.var(target) * len(target)
        sum_left = 0.0
        sum_right = np.sum(target)
        sum_sq_left = 0.0
        sum_sq_right = np.sum(target ** 2)
        n_left = 0
        n_right = len(target)
    best_gain = -np.inf
    best_threshold = None
    for i in range(1, len(feature_values)):
        if feature_values[i] == feature_values[i - 1]:
            continue  # Skip if the threshold is the same
        if is_gbdt:
            G_left += g[i - 1]
            H_left += h[i - 1]
            G_right = G_total - G_left
            H_right = H_total - H_left
            # Compute gain
            gain = 0.5 * (
                G_left ** 2 / (H_left + lamda) +
                G_right ** 2 / (H_right + lamda) -
                G_total ** 2 / (H_total + lamda)
            ) - gamma
        elif is_classification:
            # Update left and right class counts
            left_classes, left_counts = np.unique(target[:i], return_counts=True)
            right_classes, right_counts = np.unique(target[i:], return_counts=True)
            n_left = i
            n_right = len(target) - i
            gini_left = 1.0 - np.sum((left_counts / n_left) ** 2)
            gini_right = 1.0 - np.sum((right_counts / n_right) ** 2)
            # Compute weighted Gini impurity
            gini_split = (n_left / n_total) * gini_left + (n_right / n_total) * gini_right
            # Compute gain
            gain = gini_parent - gini_split
        else:
            # Update left and right sums
            val = target[i - 1]
            sum_left += val
            sum_right -= val
            sum_sq_left += val ** 2
            sum_sq_right -= val ** 2
            n_left += 1
            n_right -= 1
            # Compute variance reduction
            if n_left > 0 and n_right > 0:
                var_left = sum_sq_left - (sum_left ** 2) / n_left
                var_right = sum_sq_right - (sum_right ** 2) / n_right
                gain = total_variance - (var_left + var_right)
            else:
                gain = 0
        if gain > best_gain:
            best_gain = gain
            best_threshold = (feature_values[i] + feature_values[i - 1]) / 2
    return best_threshold, best_gain

In [ ]:
# TODO: Evaluation functions (you can use code from previous homeworks)

# RMSE
def root_mean_square_error(pred, y):
  #TODO
  rmse = np.sqrt(np.mean((pred - y) ** 2))
  return rmse

# precision
def accuracy(pred, y):
  #TODO
  pred_labels = np.round(pred)
  return np.mean(pred_labels == y)

In [24]:
# TODO: GBDT regression on boston house price dataset

# load data
import numpy as np
import pandas as pd

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]


# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Train Random Forest model
rf_model = RF(n_threads = None, loss = 'mse',
        max_depth = 6, min_sample_split = 25,
        lamda = 5, gamma = 0.5,
        rf = 0.35, num_trees = 25)

rf_model.fit(X_train, y_train)

# Make predictions on Test Data
y_pred_rf_test = rf_model.predict(X_test)

# Evaluate RMSE on Test Data
rmse_rf_test = root_mean_square_error(y_pred_rf_test, y_test)
print(f"RF Test RMSE: {rmse_rf_test}")

# Make predictions on Training Data
y_pred_rf_train = rf_model.predict(X_train)

# Evaluate RMSE on Training Data
rmse_rf_train = root_mean_square_error(y_pred_rf_train, y_train)
print(f"RF Training RMSE: {rmse_rf_train}")

# GBDT regression on Boston house price dataset

# Train GBDT model
gbdt_model = GBDT(
    n_threads=None,
    loss='mse',
    max_depth=6,
    min_sample_split=25,
    lamda=5,
    gamma=0.5,
    learning_rate=0.1,
    num_trees=25
)

gbdt_model.fit(X_train, y_train)

# Evaluate on Test Data
y_pred_gbdt_test = gbdt_model.predict(X_test)
rmse_gbdt_test = root_mean_square_error(y_pred_gbdt_test, y_test)
print(f"GBDT Test RMSE: {rmse_gbdt_test}")

# Evaluate on Training Data
y_pred_gbdt_train = gbdt_model.predict(X_train)
rmse_gbdt_train = root_mean_square_error(y_pred_gbdt_train, y_train)
print(f"GBDT Training RMSE: {rmse_gbdt_train}")

(506, 13) (506,) (354, 13) (354,) (152, 13) (152,)
RF Test RMSE: 5.286105440163269
RF Training RMSE: 4.648074557108212
GBDT Test RMSE: 4.554350406941252
GBDT Training RMSE: 3.50478513939429


In [30]:
# TODO: GBDT classification on credit-g dataset

# load data
from sklearn.datasets import fetch_openml
X, y = fetch_openml('credit-g', version=1, return_X_y=True, data_home='credit/')
y = np.array(list(map(lambda x: 1 if x == 'good' else 0, y)))

X = pd.get_dummies(X)
X = X.values

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Train GBDT model
gbdt_model = GBDT(
n_threads = None, loss = logistic(),
        max_depth = 6, min_sample_split = 25,
        lamda = 5, gamma = 0.5,
        learning_rate = 0.55, num_trees = 25
)

gbdt_model.fit(X_train, y_train)

# Make predictions on Test Data
y_pred_gbdt_test = gbdt_model.predict(X_test)

# Evaluate accuracy on Test Data
acc_gbdt_test = accuracy(y_pred_gbdt_test, y_test)
print(f"GBDT Test Accuracy on credit-g dataset: {acc_gbdt_test}")

# Make predictions on Training Data
y_pred_gbdt_train = gbdt_model.predict(X_train)

# Evaluate accuracy on Training Data
acc_gbdt_train = accuracy(y_pred_gbdt_train, y_train)
print(f"GBDT Training Accuracy: {acc_gbdt_train}")

# Train Random Forest model
rf_model = RF(
    n_threads=None,
    loss='log',  # Use 'log' loss for classification
    max_depth=6,
    min_sample_split=25,
    lamda=5,
    gamma=0.5,
    rf=0.35,
    num_trees=50
)

rf_model.fit(X_train, y_train)

# Make predictions on Test Data
y_pred_rf_test = rf_model.predict(X_test)

# Evaluate accuracy on Test Data
acc_rf_test = accuracy(y_pred_rf_test, y_test)
print(f"RF Test Accuracy: {acc_rf_test}")

# Make predictions on Training Data
y_pred_rf_train = rf_model.predict(X_train)

# Evaluate accuracy on Training Data
acc_rf_train = accuracy(y_pred_rf_train, y_train)
print(f"RF Training Accuracy: {acc_rf_train}")

(1000, 61) (1000,) (700, 61) (700,) (300, 61) (300,)
GBDT Test Accuracy on credit-g dataset: 0.6833333333333333
GBDT Training Accuracy: 0.7414285714285714
RF Test Accuracy: 0.7
RF Training Accuracy: 0.7


In [29]:
# TODO: GBDT classification on breast cancer dataset

# load data
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Train GBDT model
gbdt_model = GBDT(
    n_threads=None,
    loss='log',  # Use logistic loss for classification
    max_depth=3,
    min_sample_split=25,
    lamda=5,
    gamma=0.5,
    learning_rate=0.55,
    num_trees=25
)

gbdt_model.fit(X_train, y_train)

# Make predictions on Test Data
y_pred_gbdt_test = gbdt_model.predict(X_test)

# Evaluate accuracy on Test Data
acc_gbdt_test = accuracy(y_pred_gbdt_test, y_test)
print(f"GBDT Test Accuracy on breast-cancer dataset: {acc_gbdt_test}")

# Make predictions on Training Data
y_pred_gbdt_train = gbdt_model.predict(X_train)

# Evaluate accuracy on Training Data
acc_gbdt_train = accuracy(y_pred_gbdt_train, y_train)
print(f"GBDT Training Accuracy: {acc_gbdt_train}")

# Train Random Forest model
rf_model = RF(
    n_threads=None,
    loss='log',  # Use 'log' loss for classification
    max_depth=6,
    min_sample_split=25,
    lamda=5,
    gamma=0.5,
    rf=0.35,
    num_trees=50
)

rf_model.fit(X_train, y_train)

# Make predictions on Test Data
y_pred_rf_test = rf_model.predict(X_test)

# Evaluate accuracy on Test Data
acc_rf_test = accuracy(y_pred_rf_test, y_test)
print(f"RF Test Accuracy: {acc_rf_test}")

# Make predictions on Training Data
y_pred_rf_train = rf_model.predict(X_train)

# Evaluate accuracy on Training Data
acc_rf_train = accuracy(y_pred_rf_train, y_train)
print(f"RF Training Accuracy: {acc_rf_train}")

(569, 30) (569,) (398, 30) (398,) (171, 30) (171,)
GBDT Test Accuracy on breast-cancer dataset: 0.9590643274853801
GBDT Training Accuracy: 0.9949748743718593
RF Test Accuracy: 0.6140350877192983
RF Training Accuracy: 0.6331658291457286
